In [1]:
import pickle
import gzip
import pandas as pd

In [2]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/mid_data/pad_X_train_mid.pickle','rb') as f:
    pad_X_train_mid = pickle.load(f)

In [3]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/mid_data/pad_X_val_mid.pickle','rb') as f:
    pad_X_val_mid = pickle.load(f)

In [4]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/mid_data/del_y_train_mid.pickle','rb') as f:
    del_y_train_mid = pickle.load(f)

In [5]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/mid_data/Y_val_mid.pickle','rb') as f:
    Y_val_mid = pickle.load(f)

In [27]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigval_y.pickle','rb') as f:
    bigval_y = pickle.load(f)

In [28]:
bigval_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
Y_val_mid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 새 섹션

**모델 1**

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
import os
tf.random.set_seed(777)

In [11]:
path = "/content/drive/MyDrive/애쓰는 감자/코드/ysb/weight"
vocab_size= 18642
max_len = 300

In [18]:
model1 = Sequential()
model1.add(Embedding(vocab_size,64, input_length=max_len))
model1.add(Conv1D(64, 5, activation = 'relu', padding = 'same', kernel_regularizer=l2(0.01), kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02)))
model1.add(Conv1D(64, 5, activation = 'relu', padding = 'same', kernel_regularizer=l2(0.01), kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02)))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(77, activation = 'softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02)))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 64)           1193088   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 300, 64)           20544     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 300, 64)           20544     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 77)                5005      
Total params: 1,239,181
Trainable params: 1,239,181
Non-trainable params: 0
_________________________________________________________________


In [19]:
model1.compile(optimizer=RMSprop(lr = .0005), loss='categorical_crossentropy', metrics=['acc'])
reLR = ReduceLROnPlateau(patience = 5, verbose=1, factor = .2)
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint(filepath = os.path.join(path, 'model1_mid'), monitor='loss', mode='min', verbose=1, save_best_only=True, restore_best_weights = True)

In [20]:
val_rate = 0.2

In [21]:
history1 = model1.fit(pad_X_train_mid, del_y_train_mid, epochs=4, batch_size= 64, shuffle = True, validation_split=val_rate, verbose = 1, callbacks=[es, mc, reLR])

Epoch 1/4
14245/14245 [==============================] - 186s 11ms/step - loss: 3.1065 - acc: 0.2723 - val_loss: 1.7425 - val_acc: 0.5847

Epoch 00001: loss improved from inf to 2.50234, saving model to /content/drive/MyDrive/애쓰는 감자/코드/ysb/weight/model1_mid
INFO:tensorflow:Assets written to: /content/drive/MyDrive/애쓰는 감자/코드/ysb/weight/model1_mid/assets
Epoch 2/4
14245/14245 [==============================] - 157s 11ms/step - loss: 1.6142 - acc: 0.6194 - val_loss: 1.4356 - val_acc: 0.6574

Epoch 00002: loss improved from 2.50234 to 1.54604, saving model to /content/drive/MyDrive/애쓰는 감자/코드/ysb/weight/model1_mid
INFO:tensorflow:Assets written to: /content/drive/MyDrive/애쓰는 감자/코드/ysb/weight/model1_mid/assets
Epoch 3/4
14245/14245 [==============================] - 154s 11ms/step - loss: 1.4054 - acc: 0.6642 - val_loss: 1.3467 - val_acc: 0.6744

Epoch 00003: loss improved from 1.54604 to 1.37904, saving model to /content/drive/MyDrive/애쓰는 감자/코드/y

In [30]:
model1 = load_model( os.path.join(path, 'model1_mid')) #0.6866

In [31]:
pred_val = model1.predict(pad_X_val_mid)

In [24]:
pred_val

array([[3.40937229e-04, 1.38748355e-08, 3.61838215e-06, ...,
        7.56306573e-09, 7.31060412e-09, 1.53450426e-08],
       [1.54976873e-03, 2.20136659e-04, 2.66594365e-02, ...,
        6.03666062e-10, 6.12660367e-10, 7.75990883e-10],
       [8.35358787e-06, 5.96894978e-09, 2.83755277e-08, ...,
        1.62026337e-10, 1.66738429e-10, 3.05546199e-10],
       ...,
       [3.33967137e-06, 4.55690932e-13, 9.59587965e-07, ...,
        3.15277870e-09, 3.02703973e-09, 6.76703404e-09],
       [1.59670435e-05, 6.14384453e-06, 1.07319993e-05, ...,
        8.09863149e-11, 8.00220376e-11, 1.01284751e-10],
       [1.74756087e-02, 1.59856327e-07, 1.80756382e-04, ...,
        3.86229138e-09, 4.04264666e-09, 7.92944288e-09]], dtype=float32)

In [32]:
import numpy as np


score_board = []
for true,pred in zip(Y_val_mid,pred_val):
  if np.argmax(true) == np.argmax(pred):
    score_board.append(1)
  else:
    score_board.append(0)
print('Accuracy : ',np.array(score_board).sum()/len(score_board) * 100)

Accuracy :  33.71786602782641


In [ ]:
pred_val

array([[6.2289787e-04, 3.6714700e-05, 1.2646578e-02, ..., 7.4766495e-04,
        5.7161209e-09, 1.0791697e-08],
       [2.5530215e-03, 1.2853582e-03, 7.2891372e-01, ..., 2.7539048e-04,
        2.0227700e-08, 4.0912003e-08],
       [7.2937325e-04, 1.4042982e-04, 1.1599829e-01, ..., 3.9999833e-04,
        2.5468111e-08, 5.9669659e-08],
       ...,
       [1.3265924e-02, 2.6684414e-04, 1.5715522e-01, ..., 9.8542515e-03,
        4.0468976e-08, 9.5263914e-08],
       [6.9338066e-04, 2.7679444e-05, 9.2927116e-01, ..., 1.1654475e-03,
        6.6199028e-09, 1.5658502e-08],
       [7.2645949e-04, 1.4986157e-03, 7.8593541e-05, ..., 1.5742613e-04,
        2.1555788e-11, 6.7800397e-11]], dtype=float32)

In [ ]:
pred_val = model1.predict(pad_X_train)

In [ ]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/ysb_predict_data/m1_pred.pickle', 'wb') as f:
    pickle.dump(pred_val, f)